In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip


# LwA Dev User infos
C_KEY = 'amzn1.application-oa2-client.c..........82777b0b3'
C_SECRET_KEY = 'd12827396da..............a767c4'
C_REFRESH_TOKEN = 'Atzr|IwE.......SnTEaO'

ProfileId_DE = '216......4659'
ProfileId_IT = '215......123194'
ProfileId_FR = '118.....966079'
ProfileId_NL = '371.....4632014'
ProfileId_ES = '121.....53006'
ProfileId_UK = '3411......77547'   

In [0]:
# Get Access Token
def get_access_token():
    global refresh_token
    url = "https://api.amazon.co.uk/auth/o2/token"

    body = {
            "grant_type":"refresh_token",
            "client_id": C_KEY,
            "client_secret": C_SECRET_KEY,
            "refresh_token":C_REFRESH_TOKEN
           }
 
    response = requests.post(url = url, data = body)
    return response.json()['access_token']

C_ACCESS_TOKEN = get_access_token()

In [0]:
# Function to Request the Portfolios
def portfolio(ProfileId):
    global C_ACCESS_TOKEN, C_KEY
    url = "https://advertising-api-eu.amazon.com/v2/portfolios"

    headers = {
                'Authorization'                   : 'Bearer ' + C_ACCESS_TOKEN,
                'Amazon-Advertising-API-ClientId' :  C_KEY,
                'Amazon-Advertising-API-Scope'    :  ProfileId,
                'Content-Type'                    : 'application/json'
                }


    response = requests.get(url = url, headers = headers)
    response = response.json()
    file = json.dumps(response)
    return file

In [0]:
# Get the responses
file_DE = portfolio(ProfileId_DE)
file_IT = portfolio(ProfileId_IT)
file_FR = portfolio(ProfileId_FR)
file_NL = portfolio(ProfileId_NL)
file_ES = portfolio(ProfileId_ES)
file_UK = portfolio(ProfileId_UK)

In [0]:
# Connect to Blob Storage and Mounting. 
# sas token can be taken from the container settings > shared access tokens.
# Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error. 
containerName = "amazon-portfolio"
storageAccountName = "advertising"
sas = "sp=racwdli&st=2022......EwV4%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amazon-portfolio")
except:
    pass
    
try:
    dbutils.fs.mount( source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
                      mount_point = "/mnt/amazon-portfolio",
                      extra_configs = {config : sas}
                    )
except:
    pass

In [0]:
# Put files to the blob container
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__DE' + '.json', file_DE, overwrite = True)
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__IT' + '.json', file_IT, overwrite = True)
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__FR' + '.json', file_FR, overwrite = True)
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__NL' + '.json', file_NL, overwrite = True)
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__ES' + '.json', file_ES, overwrite = True)
dbutils.fs.put('dbfs:/mnt/amazon-portfolio/' + 'Ads' + '__UK' + '.json', file_UK, overwrite = True)

Wrote 16149 bytes.
Wrote 3716 bytes.
Wrote 4431 bytes.
Wrote 669 bytes.
Wrote 2569 bytes.
Wrote 3141 bytes.
Out[45]: True